In [ ]:
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
import os
from datetime import datetime
import matplotlib.pyplot as plt
import math

pd.options.display.max_columns = 3000
pd.options.display.max_rows = 3000

In [ ]:
acc_df = pd.read_csv('../input/features-imu-preprocessing-clean/cleaned/imu/imu_acc_train.csv')
mag_df = pd.read_csv('../input/features-imu-preprocessing-clean/cleaned/imu/imu_magnet_train.csv')
# wypt_df = pd.read_csv('../input/data-pre-processing-to-csv/cleaned/waypoint_train.csv')
# wifi_df = pd.read_csv('../input/data-pre-processing-to-csv/cleaned/wifi_train.csv')

# Accelerometer - calculate velocity 
references: https://www.kaggle.com/iamleonie/intro-to-indoor-location-navigation

https://www.kaggle.com/museas/estimate-the-walk-with-acce-and-magn/comments

accelerometer measures change in velocity 

unit: m/s^2 

mean of z = gravity 

Equation: https://www.calculatorsoup.com/calculators/physics/displacement_v_a_t.php

1. convert timestamp to seconds: ms/1000
2. calculte time difference delta
3. convert time delta to seconds
4. velocity = acceleration * timedelta in seconds
5. position = p_0 + velocity * timedelta in seconds

* calculate position and velocity from acceleration = calculate acceleration and velocity from positon

* But since wypt and accelerometer data won't match (has lots of NaN and only few intersections), integrating over the acceleration will also integrate the error for each sample which can quickly accumulate the cause large deviations. 

* Thus, can use velocity as a feature in pre-processing (as to calculate velocity we don't need position)

In [ ]:
def calc_from_acce(timestamp, acce, p_0):
    df = pd.DataFrame({'timestamp' : timestamp, 'acceleration' : acce})
    df['timestamp_ms'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000.0))
    df['timedelta_ms'] = df['timestamp_ms'].diff()
    df['timedelta_s'] = df['timedelta_ms'].apply(lambda x: x.total_seconds()).fillna(0)
    df['velocity'] = (df['acceleration']*df['timedelta_s']).cumsum()
    df['position'] = p_0 + (df['velocity']*df['timedelta_s']).cumsum()
    return df[['timestamp', 'timestamp_ms', 'timedelta_s', 'position', 'velocity', 'acceleration']]

In [ ]:
# referenced from https://www.kaggle.com/iamleonie/intro-to-indoor-location-navigation
# and made some edits 
def calc_velocity(timestamp, acce):
    df = pd.DataFrame({'timestamp' : timestamp, 'acceleration' : acce})
    df['timestamp_ms'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000.0))
    df['timedelta_ms'] = df['timestamp_ms'].diff()
    df['timedelta_s'] = df['timedelta_ms'].apply(lambda x: x.total_seconds()).fillna(0)
    df['velocity'] = (df['acceleration']*df['timedelta_s'])
    #df['position'] = p_0 + (df['velocity']*df['timedelta_s']).cumsum()
    return df[['timestamp', 'timestamp_ms', 'timedelta_s', 'velocity', 'acceleration']]

In [ ]:
acc_df = acc_df.sort_values(['building_id', 'floor', 'path_id', 'Time'])
acc_df = acc_df[['building_id','floor','path_id', 'Time','acce_x','acce_y','acce_z','acce_accuracy']]
acc_df.head()

In [ ]:
# Try sample accelerometer
acc_samp = acc_df[(acc_df['building_id'] == '5c3c44b80379370013e0fd2b') & 
                  (acc_df['floor'] == -1) & 
                  (acc_df['path_id']=='5d0761b54cae4f000a2db567')]

acc_sampe_v = calc_velocity(acc_samp['Time'],(-1)*acc_samp['acce_x'])
acc_sampe_v.drop_duplicates()

## Accelerometer - infer Number of steps

The idea of infer # of steps from accelerometer was from this notebook
https://www.kaggle.com/museas/estimate-the-walk-with-acce-and-magn/comments
, which the notebook refers from a github focuses on research on step detection algorithm https://github.com/danielmurray/adaptiv


General idea:
1. using accelerometer define crest and trough
2. calculate the distance between crest to trough 
3. define a treshold = how large the distence would you define a single step 



Limitation: as mentioend in the github, the crossings varies person by person, thus hard to define a treshold 


default treshold: crossing mean * 1.1



Definitions used: 
1. mixed all three accelerometers: x, y, z 
2. used butterworth highpass filter to eliminate low frequency components and generate a smaller (less noised) mixed accelerometer 
3. calculate crossing
4. determine number of steps 


In [ ]:
def mix_acce(timestamp,x,y,z):
    """This function mixes the three accelerometers"""
    mix_acce = np.sqrt(x**2 + y**2 + z**2)
#     mix_acce = np.concatenate(timestamp, mix_acce, 1)
    mix_acce = pd.DataFrame(pd.np.column_stack([timestamp,mix_acce]))
    mix_df = pd.DataFrame(mix_acce)
    mix_df.columns = ["timestamp","acce"]
    return mix_df

In [ ]:
mix_acce = mix_acce(acc_samp['Time'],acc_samp['acce_x'],acc_samp['acce_y'],acc_samp['acce_z'])
mix_acce

In [ ]:
from scipy.signal import butter, lfilter

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

order = 3
fs = 50.0  # sample rate, Hz
# fs = 100
# cutoff = 3.667  # desired cutoff frequency of the filter, Hz
cutoff = 3

# filtered = butter_lowpass_filter(mix_df["acce"], cutoff, fs, order)
# filtered

In [ ]:
acc_filter = butter_lowpass_filter(mix_acce['acce'],cutoff,fs,order)
acc_filter

In [ ]:
# referenced from https://www.kaggle.com/museas/estimate-the-walk-with-acce-and-magn/comments
# and made some edits 
def peak_accel_threshold(data, timestamps, threshold):
    """ This function used to calculate the crossings (difference between crest and trough) based on certain threshold"""
    d_acc = []
    last_state = 'below'
    # below - less than threshold
    # above - above the threshold
    crest_troughs = 0
    crossings = []

    for i, datum in enumerate(data):
        
        current_state = last_state
        if datum < threshold:
            current_state = 'below'
        elif datum > threshold:
            current_state = 'above'

        if current_state is not last_state:
            if current_state is 'above':
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)
            else:
                crossing = [timestamps[i], threshold]
                crossings.append(crossing)

            crest_troughs += 1
        last_state = current_state

    return np.array(crossings)

In [ ]:

threshold = acc_filter.mean() * 1.1
print(threshold)
crossings = peak_accel_threshold(acc_filter, mix_acce["timestamp"], threshold)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 5))
plt.plot(mix_acce["timestamp"], acc_filter, 'b-', color='cornflowerblue')
plt.plot(crossings.T[0], crossings.T[1], 'ro', linewidth=2)
plt.xlabel('Time [sec]')
plt.grid()
plt.legend()
plt.show()

print("sum of crossings: ", len(crossings))
print("infer number of steps: ", len(crossings)/5/10)
 

In [ ]:
# try another path 

# Using Magneic field to infer direction (calculating the euler angles) use as a compass

notebook referenced: https://www.kaggle.com/museas/estimate-the-walk-with-acce-and-magn/comments

Euler Angle Equation: https://www.researchgate.net/figure/The-Euler-angles-Roll-Pitch-and-Yaw-Prior-to-magnetic-compensation-the-recorded_fig1_275771788

Euler Angle Definition: Euler Angle describes the orientation of a rigid body with respect to a fixed coordinate system. It can also represents the orientation of a mobile frame of reference in physics or the orientation of a general basis in 3-dimensional linear algebra

Definition of roll, pitch: https://howthingsfly.si.edu/flight-dynamics/roll-pitch-and-yaw

References: https://math.stackexchange.com/questions/2466949/get-magnetic-field-values-from-euler-angle

General idea: 

1. Use acceleromer to calculate roll: the theta angel between ay, az
2. Use accelerometer to calculate pitch
3. Define m_trans (a threshold we have to define )
4. calcule euler angel based on equation 

In [ ]:
mag_df = mag_df[['building_id','floor','path_id','Time','magnet_x','magnet_y','magnet_z']]
acc_df = acc_df[['building_id','floor','path_id','Time','acce_x','acce_y','acce_z']]

In [ ]:
mag_samp = mag_df[mag_df['path_id']=='5dc7f1951cda370006031a78']

In [ ]:
acc_samp = acc_df[acc_df['path_id'] == '5dc7f1951cda370006031a78']

In [ ]:
del mag_df, acc_df

In [ ]:
print(acc_samp.shape)
print(mag_samp.shape)

In [ ]:
# even just one path subset fails.... 
acc_mag = pd.merge(mag_samp,acc_samp,how='inner',
                   on=['building_id', 'floor', 'path_id', 'Time']).sort_values(['building_id', 'floor', 'path_id', 'Time'])

In [ ]:
acc_mag.shape

In [ ]:
# Tried using outer but the notebook crasshed. 
# used inner crashed again.... still crashed 
# tried using just one path ... still crashed...
# Find out path that are both existed in magneticfield and accelerometer --- to pick samples 
# import itertools
# path_acc = set(acc_df['path_id'])
# path_mag = set(mag_df['path_id'])
# path = path_acc.intersection(path_mag)
#path
# '5dce74bb94e49000061250ca' --- failed
#  '5dc7f1951cda370006031a78',
#  '5da6e6b0ab074400064325d4',
#  '5dada49baa1d300006faaa08',
#  '5d074cbab53a8d0008dd4928',
#  '5dc7887317ffdd0006f11ed5',
#  '5da9aae7df065a00069be7f5',
#  '5da833e44091ff000604890d',
#  '5dd23c4494e4900006126ae4',
#  '5da54fbb6b51bc0006438eb8',
#  '5dd10a5594e4900006126256',
#  '5dc520fd21dceb00061147ed',

In [ ]:
acc_mag

In [ ]:
test = acc_mag.drop_duplicates()
print(test.shape)

In [ ]:
acc_mag.iloc[1][4]

In [ ]:
# Edited from https://www.kaggle.com/museas/estimate-the-walk-with-acce-and-magn/comments
def calc_direction(mag,acc):
    """This Function calculates the direction the surveyor is facing at each timestamp using euler angle """
    m_trans = -5 #0
    time_di_list = []
    df = pd.merge(mag,acc,how='inner',
                   on=['building_id', 'floor', 'path_id', 'Time']).sort_values(['building_id', 'floor', 'path_id', 'Time'])
    print(df.shape)
    for i in df.iterrows():
        gx,gy,gz = i[1][4],i[1][5],i[1][6]
        ax,ay,az = i[1][7],i[1][8],i[1][9]

        roll = math.atan2(ay,az)
        pitch = math.atan2(-1*ax , (ay * math.sin(roll) + az * math.cos(roll)))

        q =  m_trans - math.degrees(math.atan2(
                                (gz*math.sin(roll)-gy*math.cos(roll)),(gx*math.cos(pitch) + gy*math.sin(roll)*math.sin(pitch) 
                                                                       + gz*math.sin(pitch)*math.cos(roll))
                                )) -90
        if q <= 0:
            q += 360
        
        time_di_list.append((i[1][3],q))
    d_list = [x[1] for x in time_di_list]
    print(len(d_list))
        
    df['Direction'] = d_list
    return df

In [ ]:
direction_samp = calc_direction(mag_samp,acc_samp)
direction_samp

In [ ]:
dir_nodup = direction_samp.drop_duplicates(subset=['Time'])
dir_nodup

In [ ]:
# Visualize 
# Copied from Github Helper function: https://github.com/location-competition/indoor-location-competition-20/blob/master/visualize_f.py
def visualize_trajectory(trajectory, floor_plan_filename, width_meter, height_meter, title=None, mode='lines + markers + text', show=False):
    """
    Copied from from https://github.com/location-competition/indoor-location-competition-20/blob/master/visualize_f.py

    """
    fig = go.Figure()

    # add trajectory
    size_list = [6] * trajectory.shape[0]
    size_list[0] = 10
    size_list[-1] = 10

    color_list = ['rgba(4, 174, 4, 0.5)'] * trajectory.shape[0]
    color_list[0] = 'rgba(12, 5, 235, 1)'
    color_list[-1] = 'rgba(235, 5, 5, 1)'

    position_count = {}
    text_list = []
    for i in range(trajectory.shape[0]):
        if str(trajectory[i]) in position_count:
            position_count[str(trajectory[i])] += 1
        else:
            position_count[str(trajectory[i])] = 0
        text_list.append('        ' * position_count[str(trajectory[i])] + f'{i}')
    text_list[0] = 'Start 0'
    text_list[-1] = f'End {trajectory.shape[0] - 1}'

    fig.add_trace(go.Scatter(x=pred_px, y=pred_py, name="pred"))
    
    fig.add_trace(
        go.Scattergl(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            mode=mode,
            marker=dict(size=size_list, color=color_list),
            line=dict(shape='linear', color='lightgrey', width=3, dash='dash'),
            text=text_list,
            textposition="top center",
            name='trajectory',
        ))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=800,
        height=  800 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig

def visualize_train_trajectory(path):
    """
    Edited from 
    https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis
    """
    _id, floor = path.split("/")[:2]
    
    train_floor_data = read_data_file(f"../input/indoor-location-navigation/train/{path}")
    with open(f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)

    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3], 
        f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
        f"Visualization of {path}"
    )
#visualize_train_trajectory(a)

In [ ]:
build_id = '5d2709e003f801723c32d896'
floor = 'F4'
path_id = '5dc7f1951cda370006031a78'
p = build_id + '/' + floor + '/' + path_id + '.txt'
p

In [ ]:
!git clone https://github.com/location-competition/indoor-location-competition-20.git

In [ ]:
%cd /kaggle/working/indoor-location-competition-20/

In [ ]:
from io_f import read_data_file

In [ ]:
# visualize_train_trajectory('5cd56b77e2acfd2d33b5b22b/F5/5d09fa831a9498000952a1a4.txt')